Add in the spatial data



In [2]:
!pip install geopandas
!pip install pyshp

!sudo apt-get update && apt-get install -y libspatialindex-dev
!pip install rtree
!pip install geopandas

import pandas as pd
from zipfile import ZipFile
import geopandas as gpd
from io import BytesIO
import shapefile
from shapely.geometry import shape  
import rtree
import os

  Using cached geopandas-0.7.0-py2.py3-none-any.whl (928 kB)
  Using cached pyproj-2.6.1.post1-cp37-cp37m-win_amd64.whl (17.2 MB)
  Using cached Shapely-1.7.0-cp37-cp37m-win_amd64.whl (1.0 MB)
  Using cached Fiona-1.8.13.post1.tar.gz (1.2 MB)


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\natalielam\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\NATALI~1\\AppData\\Local\\Temp\\pip-install-l3wqcd1m\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\NATALI~1\\AppData\\Local\\Temp\\pip-install-l3wqcd1m\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\NATALI~1\AppData\Local\Temp\pip-install-l3wqcd1m\fiona\pip-egg-info'
         cwd: C:\Users\NATALI~1\AppData\Local\Temp\pip-install-l3wqcd1m\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Chec

'sudo' is not recognized as an internal or external command,
operable program or batch file.


  Using cached geopandas-0.7.0-py2.py3-none-any.whl (928 kB)
  Using cached pyproj-2.6.1.post1-cp37-cp37m-win_amd64.whl (17.2 MB)
  Using cached Fiona-1.8.13.post1.tar.gz (1.2 MB)


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\natalielam\Anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\NATALI~1\\AppData\\Local\\Temp\\pip-install-7fcfby0j\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\NATALI~1\\AppData\\Local\\Temp\\pip-install-7fcfby0j\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\NATALI~1\AppData\Local\Temp\pip-install-7fcfby0j\fiona\pip-egg-info'
         cwd: C:\Users\NATALI~1\AppData\Local\Temp\pip-install-7fcfby0j\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Chec

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
df = pd.read_csv('all.csv')

zipfile = ZipFile('planning-area-census2010-shp.zip')
filenames = [y for y in sorted(zipfile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 

dbf, prj, shp, shx = [BytesIO(zipfile.read(filename)) for filename in filenames]
r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)

attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]  
for row in r.shapeRecords():  
    geometry.append(shape(row.shape.__geo_interface__))  
    attributes.append(dict(zip(field_names, row.record)))

In [ ]:
gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = 'epsg:3414')
gdf.geometry = gdf.geometry.to_crs(epsg=4326)

points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.origin_lng, df.origin_lat))

points = points.loc[points.geometry.is_valid]
gdf = gdf.loc[gdf.geometry.is_valid]

In [ ]:
complete = gpd.sjoin(points, gdf, op = "within")

df_complete = pd.DataFrame(complete)

df_complete = df_complete[['trj_id', 
                           'avg_speed', 
                           'avg_bearing', 
                           'osname',
                           'hour',
                           'day',
                           'is_Weekday',
                           'time_group',
                           'origin_lat',
                           'origin_lng',
                           'dest_lat',
                           'dest_lng',
                           'duration',
                           'euclid_dist',
                           'PLN_AREA_N',
                           'REGION_N']]

df_complete['origin_subregion'] = df_complete['PLN_AREA_N']
df_complete['origin_region'] = df_complete['REGION_N']
del df_complete['PLN_AREA_N']
del df_complete['REGION_N']

points_dest = gpd.GeoDataFrame(df_complete, geometry=gpd.points_from_xy(df_complete.dest_lng, df_complete.dest_lat))

another = gpd.sjoin(points_dest, gdf, op = 'within')
df_updated = pd.DataFrame(another)

df_updated = df_updated[['trj_id', 
                         'avg_speed', 
                         'avg_bearing', 
                         'osname', 
                         'hour',
                         'day', 
                         'is_Weekday',
                         'time_group', 
                         'origin_lat', 
                         'origin_lng', 
                         'dest_lat', 
                         'dest_lng', 
                         'duration', 
                         'euclid_dist', 
                         'origin_subregion', 
                         'origin_region',
                         'PLN_AREA_N', 
                         'REGION_N']]

df_updated['dest_subregion'] = df_updated['PLN_AREA_N']
df_updated['dest_region'] = df_updated['REGION_N']
del df_updated['PLN_AREA_N']
del df_updated['REGION_N']

In [ ]:
# writing to csv
df_updated.to_csv('all_updated.csv')